In [1]:
import pandas as pd 
import torch  
import nltk

from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

C:\Users\kesha\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_ds = pd.read_csv("Dataset/cnn_dailymail/train.csv")
train_ds.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [3]:
len(train_ds)

287113

In [4]:
sampled_df = train_ds.sample(n=10000, random_state=42)

In [5]:
train_df, val_df = train_test_split(sampled_df, test_size=0.3, random_state=42)

len(train_df), len(val_df)

(7000, 3000)

In [6]:
train_dataset = Dataset.from_pandas(train_df[['article', 'highlights']]) 
val_dataset = Dataset.from_pandas(val_df[['article', 'highlights']])

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(dataset):
    return tokenizer(dataset["article"], truncation=True, padding="max_length", max_length=512)

tokenized_train_df = train_dataset.map(tokenize, batched=True, batch_size=32)
tokenized_val_df = val_dataset.map(tokenize, batched=True, batch_size=32)

Map: 100%|██████████████████████████████████████████████████████████████████| 3000/3000 [00:44<00:00, 67.63 examples/s]


In [8]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_df,
    eval_dataset=tokenized_val_df,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\kesha\anaconda3\envs\venv\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.